In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import pandas as pd
from sklearn import manifold
import matplotlib.colors as colors

# Importation des données

Les données étudiées dans ce TP provienne de la plateforme de données [Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/12379), elles contiennent des indices de similarité entre les votes des pays à l'ONU. 

- Télécharger les données de vote à l'ONU enregistrées dans le fichier [AffinitesVotesONU.txt](http://www.lsta.upmc.fr/ADDA/DataBertrand/AffinitesVotesONU.txt) (80 Mb) ainsi que le fichier [StateList.csv](http://www.lsta.upmc.fr/ADDA/DataBertrand/StateList.csv).
- Importer les données dans Python à l'aide la libraire Pandas.

Le fichier AffinitesVotesONU.txt est organisé comme indiqué ci-dessous :
- Une ligne renseigne la similarité des votes pour deux pays pendant une année.
- Les états sont codés par les champs "statea" et "stateb". 
- Le champ  "year" donne l'année considérée pour la comparaison des votes.
- Plusieurs indices de comparaison des votes sont fournis, dans la suite on s'intéresse à l'indice de similarité sur 3 niveaux  de votes (oui / non / abstension) :  "agree3un".

Afficher les premières lignes du dataframe VotingFull.

Le fichier StateList.csv est organisé comme indiqué ci-dessous :
- Une ligne correspond à un pays.
- Le champ "StateAbb" nous donne les abréviations (utile pour les représentations graphiques)
- Le champ "state" est le même code que celui utilisé dans VotingFull
- Le champ "CodeTP" est simplement le numéro de ligne
- Les champs "OPEP" et "UE" renseignent l'entrée dans l'OPEP ou dans l'UE (0 sinon).
Afficher les premières lignes du dataframe StateList.

In [ ]:
path_affinity= "/Users/bmichel/Dropbox/Enseignements/ADD-M2Stat/Tps-Notebooks/Datasets/Voting-ONU/AffinitesVotesONU.txt"
VotingFull = pd.read_csv(path_affinity,delim_whitespace=True,header=0)
path_affinity= "/Users/bmichel/Dropbox/Enseignements/ADD-M2Stat/Tps-Notebooks/Datasets/Voting-ONU/StateList.csv"
ListCountry = pd.read_csv(path_affinity,header=0)

Affichage des premières lignes de VotingFull et ListCountry

In [ ]:
VotingFull.head()

In [ ]:
ListCountry.head()

# Extraction des similarités de vote pour une periode donnée

In [ ]:
# periode étudiée :
annee_debut = 2000
annee_fin  = 2015  

# indice max des indices "state" (.values : syntaxe numpy )
maxstate = max(ListCountry.state.values)

# indice max dans CodeTP
maxCodeTP = max(ListCountry.CodeTP.values)

Le vecteur Translate ci-dessous permet de passer du codage "state" au codage "CodeTP".

In [ ]:
Translate =   - np.ones(maxstate+1,dtype = int) #
for numline in range(maxCodeTP):
    Translate[ListCountry.state[numline]] =  ListCountry.CodeTP[numline]

Par exemple le pays de code statea = 40 a pour indice indice 3 (on part de 0) pour "CodeTP" :

In [ ]:
Translate[40]

### Création de la matrice de dissimilarité $Dissimil$ 

Les numéros de ligne et de colonnes de $Dissimil$ correspondent au code "CodeTP". On remplit la matrice de similarité en parcourant les lignes de VotingFull, en ne selectionant que l'année "annee_choice" (ce code peut prendre quelques secondes).

La dissimilarité d'une année t est définie par :  1 - agree3un(t).

On cumule les dissimilarités sur la periode étudiée : 
$$Dissimil (paysA,paysB) = \sum_{t= debut} ^{fin}  (1- agree3un(t)) $$ 

In [ ]:
# création de la matrice (de bonne dimension)
# que des 0 partout par défaut

Dissimil = np.zeros((maxCodeTP+1,maxCodeTP+1)) 

# boucle sur les lignes de VotingFull (prend quelques secondes ...)
for numline in range(VotingFull.shape[0]):
    if (VotingFull.year[numline] >= annee_debut)&(VotingFull.year[numline] <= annee_fin): 
        Dissimil[Translate[VotingFull.statea[numline]],Translate[VotingFull.stateb[numline]]] +=  1-VotingFull.agree3un[numline]
        Dissimil[Translate[VotingFull.stateb[numline]],Translate[VotingFull.statea[numline]]] +=  1- VotingFull.agree3un[numline]

In [ ]:
Dissimil

Attention !  Certaines lignes n'ont pas été remplies. En effet tous les pays ne sont pas renseignés dans la matrice car certains pays n'existent pas ou plus à la date choisie. 
Par exemple : 

In [ ]:
Dissimil[50,]

On doit retirer ces pays de la matrice de similarité, ce sont les pays dont les lignes ou colonnes ont une somme égale à - maxCodeTP (car la matrice est de taille maxCodeTP x maxCodeTP).

In [ ]:
I = np.array([ i for i in range(maxCodeTP) if Dissimil.sum(1)[i] != 0])
print(I)

Extraction de la sous matrice de similarité :

In [ ]:
DissimilExtract   = Dissimil[I,:][:,I]  #  rq : Dissimil[I,I] ne fonctionne pas

Il faut aussi remplacer les -1  par des 0 sur la diagonale :

In [ ]:
n,p = DissimilExtract.shape

Extraction du sous tableau de  ListCountry :

In [ ]:
ListCountryExtract = ListCountry.iloc[I]
print(DissimilExtract[:,0])

# Classical MDS

**Exercice** : Ecrire une fonction Python pour l'algorithme CMDS :
- en entrée une matrice de dissimilarité et une dimension p
- en sortie : une configuration du nuage dans $\mathbb R^p$.
Ecrire cette fonction dans un fichier nommé ClassicalMDS.py, que vous pourrez utiliser dans le notebook en l'important comme suit (consultez par exemple ce [lien](https://fr.wikibooks.org/wiki/Apprendre_à_programmer_avec_Python/Fonctions_originales)) :

In [ ]:
from ClassicalMDS import *

In [ ]:
configCMDS = ClassicalMDS(Dissimil,3)
print(configCMDS)

In [ ]:
plt.figure(figsize = (15,15))
CMDSAxe0 =configCMDS[:,0]
CMDSAxe1 =configCMDS[:,1]
plt.scatter(CMDSAxe0,CMDSAxe1, marker = 'o', s =  1)
for label, x, y, OPEP,UE in zip(ListCountryExtract.StateAbb, CMDSAxe0,CMDSAxe1,ListCountryExtract.OPEP,ListCountryExtract.UE):
    plt.annotate(label, xy = (x+0.01, y))
    if OPEP > 0:
        plt.text(x+0.01,y,label, bbox={'facecolor':'red'})
    if UE > 0:
        plt.text(x+0.01,y,label, bbox={'facecolor':'blue'})        



# Clustering hierarchique en utilisant la matrice de dissimilarité

Nous allons effectuer une classification hierarchique sur les données, en utilisant la dissimilarité construite plus haut. Nous allons d'abord utiliser la librairie Sklearn et ensuite la librairie Scipy.

## Avec la fonction AgglomerativeClustering de Sklearn

Nous utilisons tout d'abord la fonction AgglomerativeClustering du module cluster de sklearn :
http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html . 

On indique à la fonction que la matrice d'affinité est déjà calculée (affinity='precomputed') car nous disposons ici de la matrice de dissimilarité calculée précédemment. La mesure d'agrégation utilisée ici est la moyenne des toutes les paires de points (linkage="average"). On impose ici 15 groupes (n_clusters=15). Faites varier le nombres de groupes et observez la distributin des clusters obtenus.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
n_clusters = 15 # nombre de clusters demandés
clustering_voting = AgglomerativeClustering(linkage="average", n_clusters=n_clusters,affinity='precomputed')
clustering_voting.fit(DissimilExtract)

Affichage du clustering obtenu :

In [ ]:
pd.set_option('display.max_rows', 200) # reglage du nombre maximal de lignes affichée pour les objets pandas.
for i in range(0,n_clusters):
    print("Classe " + str(i))
    print(ListCountryExtract.StateAbb[clustering_voting.labels_==i])
    print("---------------------")

Réprésentation du clustering obtenu sur la configuration CMDS :

In [ ]:
plt.figure(figsize=(10, 10))
for c, i, clust in zip(colors.cnames, range(n_clusters),range(n_clusters)):
    plt.scatter(CMDSAxe0[clustering_voting.labels_==i],CMDSAxe1[clustering_voting.labels_==i], c=c, label=clust)
plt.legend()

On observe un très gros cluster dominant qui capte une grande partie des pays et un seconde cluster de taille plus petite dans lequel on retrouve beaucoup de payes européens. Nous voudrions représenter le dendrogramme de la classification hierarchique pour visualiser la hierarchie, mais celui-ci n'est pas disponible dans les fonctionnalités de sklearn.

# Clustering hierarchique avec la librairie Scipy


Le module cluster de la librairie Scipy contient de nombreuses fonctions permettant de mettre en oeuvre les méthodes de classification non supervisée. Nous allons utiliser la fonction hierarchy de ce module, qui offre notamment la possibilité de tracer le dendrogramme, voir 
http://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html#module-scipy.cluster.hierarchy.

La première étape pour effectuer  une classification hierarchique en utilisant consiste à calculer la matrice de distance. Classiquement cette étape est effectuée par la fonction pdist de scipy :
http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.pdist.html
Cette fonction renvoie une matrice sous forme condensée (en effet la matrice de distance étant symétrique, il n'est pas nécessaire de stocker tous ses coefficients). La seconde étape consiste à appliquer la fonction linkage à la matrice de distance condensée
http://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.cluster.hierarchy.linkage.html

Dans le cas présent, nous avons utilisons la matrice de dissimilarité calculée auparavant, il s'agit de la matrice de dissimilarité calculée plus haut. Pour pouvoir lui appliquer la fonction linkage, il nous faut préalablement convertir la matrice de dissimilarité sous une "forme condensée" :

In [ ]:
import scipy.spatial.distance as ssd # module scipy pour manipuler des distances 
distArray = ssd.squareform(DissimilExtract)

Nous pouvons maintenant utliser la fonction linkage du module clus.hierarchy

### Average method

Il s'agit ici d'effectuer une classification hierarchique ascendante en utilisant comme mesure d'agrégation la moyenne de toutes les paires de distances (method = 'average')

In [ ]:
import scipy.cluster as clus
linkONU_av = clus.hierarchy.linkage(distArray, method='average' )

Nous allons maintenant pouvoir représenter le dendogramme de cette classification hierarchique

In [ ]:
from scipy.cluster.hierarchy import dendrogram
plt.figure(figsize=(15, 30))
plt.title('Dendogramme de la classification hierarchique, methode average')
plt.xlabel('Pays')
plt.ylabel('')
P = dendrogram(linkONU_av,
    labels = ListCountryExtract.StateNme.values,
    orientation = "left",
    leaf_font_size=8.) 

Sous pousser très en avant l'analyse du dendrogramme, on retrouve des groupes qui font sens d'un point de vue géopolitique  (notamment le groupe Européen). La structure du dendogramme explique aussi pourquoi beaucoup de clusters ne contiennent qu' un pays :  on observe un effet de chaînage quand on considère les sous-arbres.

### Single method (plus courte distance) 

In [ ]:
linkONU_single = clus.hierarchy.linkage(distArray, method='single' )
plt.figure(figsize=(15, 30))
plt.title('Dendogramme de la classification hierarchique, methode single')
plt.xlabel('Pays')
plt.ylabel('')
P = dendrogram(linkONU_single,
    labels = ListCountryExtract.StateNme.values,
    orientation = "left",
    leaf_font_size=8.) 

Remarque : Avec scipy, il est possible aussi d'extraire les groupes (en seuillant le dendrogramme à un niveau fixé) mais cela demande un peu plus de travail qu'avec Sklearn dans ce cas précis où l'on utilise une matrice de dissimilarité ad-hoc. Dans le cas plus classique où l'on observe des points dans un espace euclidien on peut facilement utiliser  la fonction fcluster de scipy.

##  Modèle de mélange Gaussiens pour le clustering

Nous allons illustrer une méthode de classification avec des modèles Gaussiens. Pour cela nous allons considérer la configuration en 2 dimension proposée par CMDS auparavent. Le module  mixture.GMM de  sklearn permet d'ajuster un modèle de mélange gaussien sur un nuage de points dans $\mathbb R ^2$ dans le but d'en déduire une classification non supervisée des données :
http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GMM.html


Configuration CMDS de dimension 2 en utilisant le script ClassicalMDS :

In [ ]:
from ClassicalMDS import *
configCMDS = ClassicalMDS(Dissimil,2)

On ajuste sur les données des modèles de mélange gaussiens de 2 à 12 composantes avec des matrices de var-cov de forme non contrainte. On calcule pour chacun d'eux le critère BIC. Le "meilleur" modèle est possède  6 composantes dans le cas présent.

In [ ]:
from sklearn import mixture
bic = []
for k in range(2,12):
    gmm = mixture.GMM(n_components=k, covariance_type='full')
    gmm.fit(configCMDS)
    bic.append(gmm.bic(configCMDS))
plt.scatter(range(2,12),bic)

In [ ]:
gmm = mixture.GMM(n_components=6, covariance_type='full')
gmm.fit(configCMDS)

Calcul de la logvraisemblance sur un maillage fin :

In [ ]:
x = np.linspace(-15.0, 10.0)
y = np.linspace(-4.0, 6.0)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -gmm.score_samples(XX)[0] # la liste des logvrais aux points de la grille XX
Z = Z.reshape(X.shape) # on reforme un tableau de meme dimension que celles de X

Représentation des iso-logvraisemblance sur ce maillage :

In [ ]:
from matplotlib.colors import LogNorm
plt.figure(figsize = (10,10))
CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=0.1, vmax=1000.0),levels=np.logspace(0, 10, 70))
CMDSAxe0 =configCMDS[:,0]
CMDSAxe1 =configCMDS[:,1]
plt.scatter(CMDSAxe0,CMDSAxe1, marker = 'o', s =  1)

 Affection des points de l'échantillon à  un groupe

In [ ]:
Groupe = gmm.predict(configCMDS)
print(Groupe)

In [ ]:
plt.figure(figsize = (10,10))
for c, i, clust in zip(colors.cnames, range(6),range(6)):
    plt.scatter(CMDSAxe0[Groupe==i],CMDSAxe1[Groupe==i], c=c, label=clust)
plt.legend()